# Table of Content

- [8.1 Changing the String Representation of Instances](#8.1)
- [8.5 Encapsulating Names in a Class](#8.5)

---
## <a name="8.1"></a> 8.1 Changing the String Representation of Instances

### Solution
Define `__str__` or `__repr__`

### Discussion
Using `format()` with `self` to implement `__repr__`  
(`!r` indicates that the output of `__repr__` should be used instead of `__str__`)

In [1]:
class Class:
    def __init__(self):
        self.x = 1
        self.y = 2
    
    def __repr__(self):
        return 'x: {0.x!r} y: {0.y}'.format(self)
    

c = Class()
print(c)

x: 1 y: 2


---
## <a name="8.5"></a> 8.5 Encapsulating Names in a Class

### Solution

- `_name`: Internal implementation
- `__name`: Private. These attributes will be renamed as `_class__name`
- `name_`: Used when `name` clashes with reserved word

### Discussion
Most time, `_name` should be enough.  
`__name` should be used when your code will involve subcalssing and these attributes should be hidden from subclasses